<a href="https://colab.research.google.com/github/changnos/2021-2-OSSP1-NotScary-2/blob/main/predict_jangdan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo pip install numba
!sudo pip install llvmlite
!sudo pip install librosa

#한글 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import tensorflow as tf
import numpy as np

file_name = '1028.h5'

model = tf.keras.models.load_model('/content/drive/MyDrive/janggu_data/소리모델/' + file_name)

In [ ]:
import os
path = '/content/drive/MyDrive/janggu_data/jangdan/name'
file_list = os.listdir(path)

for file in file_list:
    print(file[0:-5])

In [ ]:
import librosa.display, librosa
import matplotlib.pyplot as plt
import soundfile as sf
import unicodedata
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
sound_name = ['덕', '덩', '쿵']
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

accuracy = [];
def predictSound(file):
    y, sr = librosa.load(path + '/' + file)
    ans = []
    if(unicodedata.normalize('NFC',file[0:-5]) == '세마치'):
        ans = ['덩', '덩', '덕', '쿵', '덕']
    if(unicodedata.normalize('NFC',file[0:-5]) == '중중모리'):
        ans = ['덩', '덕', '쿵', '덕', '덕', '쿵', '쿵', '덕', '쿵', '쿵']
    if(unicodedata.normalize('NFC',file[0:-5]) == '휘모리'):
        ans = ['덩', '덕', '덕', '쿵', '덕', '쿵']
    if(unicodedata.normalize('NFC',file[0:-5]) == '굿거리'):
        ans = ['덩', '덕', '쿵', '덕', '쿵', '덩', '덕', '쿵', '덕', '쿵']
    if(unicodedata.normalize('NFC',file[0:-5]) == '자진모리'):
        ans = ['덩', '덕', '쿵', '덕', '쿵', '덕', '쿵', '덕']

    print(file[0:-5])
    print(ans)
    # ans = []
    # for i in range(int(int(len(file[0:-4]))/3)):
    #     ans.append(file[3*i:3*i+3])
    D = np.abs(librosa.stft(y))
    onset_env = librosa.onset.onset_strength(y=y, sr=sr,
                                            hop_length=512,
                                            aggregate=np.median)
    peaks = librosa.util.peak_pick(onset_env, pre_max=1, post_max=3, pre_avg=5, post_avg=5, delta=5, wait=1)
    D = np.abs(librosa.stft(y))
    D_db = librosa.amplitude_to_db(D, ref=np.max)
    times = librosa.times_like(onset_env, sr=sr, hop_length=512)
    tmp2 = np.array([len(times)-1])
    peaks = np.append(peaks, tmp2)
    result = []
    for i in range(len(peaks)-1):
        start_frame = peaks[i]-3
        end_frame = peaks[i + 1]-3
        ny = y[round(start_frame*512):round(end_frame*512)]
        mfcc = librosa.feature.mfcc(y=ny, sr=sr, n_mfcc=100)
        pad_width = 86 - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0), (0,pad_width)),mode='constant')
        mfcc= np.expand_dims(mfcc, 0)
        check = model.predict(mfcc)[0].tolist()
        cc = check.index(max(check))
        print(sound_name[cc])
        result.append(sound_name[cc])

    # 최대 길이 공통 부분 구하는 알고리즘 필요
    al = len(ans)
    print("al: ", al);
    cnt = 0
    for i in range(al):
        if(result[i] == unicodedata.normalize('NFC',ans[i])):
            cnt += 1
    accuracy.append(cnt/al*100);
    print(ans)
    print("정확도: " + str(cnt/al*100) + "%")
    avg = sum(accuracy)/len(accuracy)
    print("정확도 평균: " + str(avg) + "%")


In [ ]:
for file in file_list:
    predictSound(file);